In [1]:
import pandas as pd

## Repository and Data Access

1. [Fork](https://help.github.com/articles/fork-a-repo/) my [github repository](https://github.com/edeno/Jadhav-2016-Data-Analysis)

2. [Clone](https://help.github.com/articles/fork-a-repo/) the forked repository to a local directory

3. Install miniconda (or anaconda) if it isn't already installed. Type into bash:
```bash
wget https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh -O miniconda.sh;
bash miniconda.sh -b -p $HOME/miniconda
export PATH="$HOME/miniconda/bin:$PATH"
hash -r
```

3. Switch to the development branch. Type into bash:
```bash
git checkout develop
```

4. Go to the local repository (`.../Jadhav-2016-Data-Analysis`) and install the anaconda environment for the repository. Type into bash:
```bash
conda config --set always_yes yes --set changeps1 no
conda update -q conda
conda info -a
conda env create -f environment.yml
source activate Jadhav-2016-Data-Analysis
python setup.py develop
```

6. Make sure the environment is set up correctly by running the tests. Type into bash:
```bash
pytest
```

7. Copy the data folders `HPa_direct`, `HPb_direct`, `HPc_direct` from the dropbox folder `EastWestSideHippos Team Folder/HCPFCdata` to `Jadhav-2016-Data-Analysis/Raw-Data`


## Accessing the Data via Python

### Parameters Module
The `src.parameters` module has some convenient constants for accessing the data. Most important are the `ANIMALS` dictionary, the `N_DAYS`, and the `SAMPLING_FREQUENCY`.

The `ANIMALS` dictionary maps the animal name to the data directory containing the animal's data.

In [2]:
from src.parameters import ANIMALS

ANIMALS

{'HPa': Animal(short_name='HPa', directory='HPa_direct'),
 'HPb': Animal(short_name='HPb', directory='HPb_direct'),
 'HPc': Animal(short_name='HPc', directory='HPc_direct')}

`N_DAYS` corresponds to the number of days of recording and `SAMPLING_FREQUENCY` corresponds to the sampling rate of the tetrodes recording neural activity

In [3]:
from src.parameters import N_DAYS, SAMPLING_FREQUENCY

print('Days: {0}'.format(N_DAYS))
print('Sampling Frequency: {0}'.format(SAMPLING_FREQUENCY))

Days: 8
Sampling Frequency: 1500


### Data Processing Module

The `src.data_processing` module has convenient functions for importing the data files into Pandas dataframes. You can use these functions in conjunction with the constants from the `src.parameters` module. The most useful functions are:

+ `make_epochs_dataframe`: returns descriptive information about each epoch
+ `make_tetrode_dataframe`: returns descriptive information about each tetrode
+ `make_neuron_dataframe`: returns descriptive information about each neuron 
+ `get_spike_indicator_dataframe`: returns the spiking data for each neuron for each time point
+ `get_interpolated_position_dataframe`: returns data about the position of animal for each time point

Let's take each of these functions in turn.

#### `make_epochs_dataframe`

An epoch is a single recording session. There are typically multiple recording sessions (epochs) a day. The epoch dataframe gives information about what task the animal is performing during the epoch (sleeping, running a linear track, running a w-track, resting, etc). Each row of the epoch dataframe corresponds to an epoch.

In [4]:
from src.data_processing import make_epochs_dataframe

days = range(1, N_DAYS + 1)
epoch_info = make_epochs_dataframe(ANIMALS, days)
epoch_info

animal  day  epoch environment   type
animal day epoch                                      
HPa    1   1        HPa    1      1    presleep  sleep
           2        HPa    1      2         lin    run
           3        HPa    1      3         NaN   rest
           4        HPa    1      4        wtr1    run
           5        HPa    1      5         NaN   rest
           6        HPa    1      6        wtr1    run
           7        HPa    1      7   postsleep  sleep
       2   1        HPa    2      1    presleep  sleep
           2        HPa    2      2        wtr1    run
           3        HPa    2      3         NaN   rest
           4        HPa    2      4        wtr1    run
           5        HPa    2      5   postsleep  sleep
       3   1        HPa    3      1    presleep  sleep
           2        HPa    3      2        wtr1    run
           3        HPa    3      3         NaN   rest
           4        HPa    3      4        wtr1    run
           5        HPa    3      5   postsleep  sleep
       4   1        HPa    4      1    presleep  sleep
           2        HPa    4      2        wtr1    run
           3        HPa    4      3         NaN   rest
           4        HPa    4      4        wtr1    run
           5        HPa    4      5   postsleep  sleep
       5   1        HPa    5      1    presleep  sleep
           2        HPa    5      2        wtr1    run
           3        HPa    5      3         NaN   rest
           4        HPa    5      4        wtr1    run
           5        HPa    5      5   postsleep  sleep
       6   1        HPa    6      1    presleep  sleep
           2        HPa    6      2        wtr1    run
           3        HPa    6      3         NaN   rest
...                 ...  ...    ...         ...    ...
HPc    3   1        HPc    3      1    presleep  sleep
           2        HPc    3      2        wtr1    run
           3        HPc    3      3         NaN   rest
           4        HPc    3      4        wtr1    run
           5        HPc    3      5   postsleep  sleep
       4   1        HPc    4      1    presleep  sleep
           2        HPc    4      2        wtr1    run
           3        HPc    4      3         NaN   rest
           4        HPc    4      4        wtr1    run
           5        HPc    4      5   postsleep  sleep
       5   1        HPc    5      1    presleep  sleep
           2        HPc    5      2        wtr1    run
           3        HPc    5      3         NaN   rest
           4        HPc    5      4        wtr1    run
           5        HPc    5      5   postsleep  sleep
       6   1        HPc    6      1    presleep  sleep
           2        HPc    6      2        wtr1    run
           3        HPc    6      3         NaN   rest
           4        HPc    6      4        wtr2    run
           5        HPc    6      5   postsleep  sleep
       7   1        HPc    7      1    presleep  sleep
           2        HPc    7      2        wtr1    run
           3        HPc    7      3         NaN   rest
           4        HPc    7      4        wtr2    run
           5        HPc    7      5   postsleep  sleep
       8   1        HPc    8      1    presleep  sleep
           2        HPc    8      2        wtr1    run
           3        HPc    8      3         NaN   rest
           4        HPc    8      4        wtr2    run
           5        HPc    8      5   postsleep  sleep

[126 rows x 5 columns]

Often we use the epoch as a key to access more information about that epoch (information about the tetrodes, neuron, etc). Epoch keys are tuples with the following format: (Animal, Day, Epoch).

The index of the epoch dataframe can be used to produce these keys.

In [5]:
epoch_info.index.tolist()

[('HPa', 1, 1),
 ('HPa', 1, 2),
 ('HPa', 1, 3),
 ('HPa', 1, 4),
 ('HPa', 1, 5),
 ('HPa', 1, 6),
 ('HPa', 1, 7),
 ('HPa', 2, 1),
 ('HPa', 2, 2),
 ('HPa', 2, 3),
 ('HPa', 2, 4),
 ('HPa', 2, 5),
 ('HPa', 3, 1),
 ('HPa', 3, 2),
 ('HPa', 3, 3),
 ('HPa', 3, 4),
 ('HPa', 3, 5),
 ('HPa', 4, 1),
 ('HPa', 4, 2),
 ('HPa', 4, 3),
 ('HPa', 4, 4),
 ('HPa', 4, 5),
 ('HPa', 5, 1),
 ('HPa', 5, 2),
 ('HPa', 5, 3),
 ('HPa', 5, 4),
 ('HPa', 5, 5),
 ('HPa', 6, 1),
 ('HPa', 6, 2),
 ('HPa', 6, 3),
 ('HPa', 6, 4),
 ('HPa', 6, 5),
 ('HPa', 7, 1),
 ('HPa', 7, 2),
 ('HPa', 7, 3),
 ('HPa', 7, 4),
 ('HPa', 7, 5),
 ('HPa', 8, 1),
 ('HPa', 8, 2),
 ('HPa', 8, 3),
 ('HPa', 8, 4),
 ('HPa', 8, 5),
 ('HPb', 1, 1),
 ('HPb', 1, 2),
 ('HPb', 1, 3),
 ('HPb', 1, 4),
 ('HPb', 1, 5),
 ('HPb', 1, 6),
 ('HPb', 1, 7),
 ('HPb', 2, 1),
 ('HPb', 2, 2),
 ('HPb', 2, 3),
 ('HPb', 2, 4),
 ('HPb', 2, 5),
 ('HPb', 3, 1),
 ('HPb', 3, 2),
 ('HPb', 3, 3),
 ('HPb', 3, 4),
 ('HPb', 3, 5),
 ('HPb', 4, 1),
 ('HPb', 4, 2),
 ('HPb', 4, 3),
 ('HPb',

This is useful if we want to filter the epoch dataframe by a particular attribute (Say we only want sessions where the animal is asleep) and use the keys to access the data for that epoch.

In [6]:
epoch_info.loc[epoch_info.type == 'run']

animal  day  epoch environment type
animal day epoch                                    
HPa    1   2        HPa    1      2         lin  run
           4        HPa    1      4        wtr1  run
           6        HPa    1      6        wtr1  run
       2   2        HPa    2      2        wtr1  run
           4        HPa    2      4        wtr1  run
       3   2        HPa    3      2        wtr1  run
           4        HPa    3      4        wtr1  run
       4   2        HPa    4      2        wtr1  run
           4        HPa    4      4        wtr1  run
       5   2        HPa    5      2        wtr1  run
           4        HPa    5      4        wtr1  run
       6   2        HPa    6      2        wtr1  run
           4        HPa    6      4        wtr2  run
       7   2        HPa    7      2        wtr1  run
           4        HPa    7      4        wtr2  run
       8   2        HPa    8      2        wtr1  run
           4        HPa    8      4        wtr2  run
HPb    1   2        HPb    1      2         lin  run
           4        HPb    1      4        wtr1  run
           6        HPb    1      6        wtr1  run
       2   2        HPb    2      2        wtr1  run
           4        HPb    2      4        wtr1  run
       3   2        HPb    3      2        wtr1  run
           4        HPb    3      4        wtr1  run
       4   2        HPb    4      2        wtr1  run
           4        HPb    4      4        wtr1  run
       5   2        HPb    5      2        wtr1  run
           4        HPb    5      4        wtr1  run
       6   2        HPb    6      2        wtr1  run
           4        HPb    6      4        wtr2  run
       7   2        HPb    7      2        wtr1  run
           4        HPb    7      4        wtr2  run
       8   2        HPb    8      2        wtr1  run
           4        HPb    8      4        wtr2  run
HPc    1   2        HPc    1      2         lin  run
           4        HPc    1      4        wtr1  run
           6        HPc    1      6        wtr1  run
       2   2        HPc    2      2        wtr1  run
           4        HPc    2      4        wtr1  run
       3   2        HPc    3      2        wtr1  run
           4        HPc    3      4        wtr1  run
       4   2        HPc    4      2        wtr1  run
           4        HPc    4      4        wtr1  run
       5   2        HPc    5      2        wtr1  run
           4        HPc    5      4        wtr1  run
       6   2        HPc    6      2        wtr1  run
           4        HPc    6      4        wtr2  run
       7   2        HPc    7      2        wtr1  run
           4        HPc    7      4        wtr2  run
       8   2        HPc    8      2        wtr1  run
           4        HPc    8      4        wtr2  run

In [7]:
epoch_info.loc[epoch_info.type == 'run'].index.tolist()

[('HPa', 1, 2),
 ('HPa', 1, 4),
 ('HPa', 1, 6),
 ('HPa', 2, 2),
 ('HPa', 2, 4),
 ('HPa', 3, 2),
 ('HPa', 3, 4),
 ('HPa', 4, 2),
 ('HPa', 4, 4),
 ('HPa', 5, 2),
 ('HPa', 5, 4),
 ('HPa', 6, 2),
 ('HPa', 6, 4),
 ('HPa', 7, 2),
 ('HPa', 7, 4),
 ('HPa', 8, 2),
 ('HPa', 8, 4),
 ('HPb', 1, 2),
 ('HPb', 1, 4),
 ('HPb', 1, 6),
 ('HPb', 2, 2),
 ('HPb', 2, 4),
 ('HPb', 3, 2),
 ('HPb', 3, 4),
 ('HPb', 4, 2),
 ('HPb', 4, 4),
 ('HPb', 5, 2),
 ('HPb', 5, 4),
 ('HPb', 6, 2),
 ('HPb', 6, 4),
 ('HPb', 7, 2),
 ('HPb', 7, 4),
 ('HPb', 8, 2),
 ('HPb', 8, 4),
 ('HPc', 1, 2),
 ('HPc', 1, 4),
 ('HPc', 1, 6),
 ('HPc', 2, 2),
 ('HPc', 2, 4),
 ('HPc', 3, 2),
 ('HPc', 3, 4),
 ('HPc', 4, 2),
 ('HPc', 4, 4),
 ('HPc', 5, 2),
 ('HPc', 5, 4),
 ('HPc', 6, 2),
 ('HPc', 6, 4),
 ('HPc', 7, 2),
 ('HPc', 7, 4),
 ('HPc', 8, 2),
 ('HPc', 8, 4)]

#### `make_tetrode_dataframe`

`make_tetrode_dataframe` is a dictionary of pandas dataframes, where the keys corresponds to an epoch and the values are pandas dataframes detailing information about the tetrodes in that epoch (brain area, number of cells recorded, etc.).

The dictionary keys for the epochs are tuples (Animal, Day, Epoch). For example, let's load the tetrode dataframe and display all the keys:

In [8]:
from src.data_processing import make_tetrode_dataframe

tetrode_info = make_tetrode_dataframe(ANIMALS)
list(tetrode_info.keys())

[('HPa', 2, 1),
 ('HPa', 8, 2),
 ('HPa', 7, 5),
 ('HPb', 6, 4),
 ('HPb', 6, 2),
 ('HPa', 2, 2),
 ('HPa', 8, 3),
 ('HPa', 7, 4),
 ('HPc', 2, 4),
 ('HPb', 1, 2),
 ('HPa', 2, 3),
 ('HPb', 3, 4),
 ('HPc', 2, 5),
 ('HPa', 2, 4),
 ('HPa', 8, 1),
 ('HPa', 5, 5),
 ('HPa', 2, 5),
 ('HPb', 3, 2),
 ('HPa', 5, 4),
 ('HPc', 4, 4),
 ('HPb', 7, 2),
 ('HPb', 3, 3),
 ('HPc', 4, 5),
 ('HPa', 8, 4),
 ('HPb', 7, 5),
 ('HPa', 3, 5),
 ('HPa', 8, 5),
 ('HPb', 5, 4),
 ('HPa', 5, 1),
 ('HPa', 3, 4),
 ('HPb', 7, 4),
 ('HPa', 3, 3),
 ('HPc', 3, 1),
 ('HPb', 8, 4),
 ('HPa', 5, 3),
 ('HPc', 4, 1),
 ('HPa', 7, 2),
 ('HPb', 5, 1),
 ('HPb', 5, 3),
 ('HPa', 5, 2),
 ('HPa', 3, 1),
 ('HPb', 6, 1),
 ('HPb', 4, 2),
 ('HPc', 3, 3),
 ('HPc', 4, 3),
 ('HPc', 3, 2),
 ('HPb', 8, 3),
 ('HPc', 3, 5),
 ('HPb', 5, 5),
 ('HPb', 8, 2),
 ('HPb', 2, 5),
 ('HPc', 3, 4),
 ('HPb', 8, 1),
 ('HPb', 2, 4),
 ('HPc', 6, 4),
 ('HPc', 8, 2),
 ('HPb', 2, 2),
 ('HPc', 6, 5),
 ('HPb', 4, 5),
 ('HPa', 6, 4),
 ('HPc', 6, 2),
 ('HPb', 4, 4),
 ('HPa',

If we want to access a particular epoch we can just pass the corresponding epoch tuple. In this case, we want animal HPa, day 6, epoch 2. This returns a dataframe where each row corresponds to a tetrode for that epoch.

In [9]:
epoch_key = ('HPa', 6, 2)
tetrode_info[epoch_key]

area  depth descrip  numcells animal  day  \
animal day epoch tetrode_number                                              
HPa    6   2     1                CA1    114  riptet         7    HPa    6   
                 2                CA1    121     NaN         0    HPa    6   
                 3                CA1     90  CA1Ref         0    HPa    6   
                 4                CA1    117  riptet         8    HPa    6   
                 5                CA1    117  riptet         5    HPa    6   
                 6                CA1    110  riptet         0    HPa    6   
                 7                CA1    114  riptet         0    HPa    6   
                 8               iCA1    114  riptet         1    HPa    6   
                 9               iCA1    101  riptet         1    HPa    6   
                 10              iCA1     98     NaN         1    HPa    6   
                 11              iCA1    108  riptet         1    HPa    6   
                 12              iCA1    116  riptet         1    HPa    6   
                 13              iCA1    120     NaN         0    HPa    6   
                 14              iCA1    105  riptet         1    HPa    6   
                 15               PFC    120     NaN         0    HPa    6   
                 16               PFC    123     NaN         0    HPa    6   
                 17               PFC    123     NaN         1    HPa    6   
                 18               PFC    112     NaN         1    HPa    6   
                 19               PFC    130     NaN         0    HPa    6   
                 20               PFC    109     NaN         0    HPa    6   

                                 epoch  tetrode_number tetrode_id  
animal day epoch tetrode_number                                    
HPa    6   2     1                   2               1     HPa621  
                 2                   2               2     HPa622  
                 3                   2               3     HPa623  
                 4                   2               4     HPa624  
                 5                   2               5     HPa625  
                 6                   2               6     HPa626  
                 7                   2               7     HPa627  
                 8                   2               8     HPa628  
                 9                   2               9     HPa629  
                 10                  2              10    HPa6210  
                 11                  2              11    HPa6211  
                 12                  2              12    HPa6212  
                 13                  2              13    HPa6213  
                 14                  2              14    HPa6214  
                 15                  2              15    HPa6215  
                 16                  2              16    HPa6216  
                 17                  2              17    HPa6217  
                 18                  2              18    HPa6218  
                 19                  2              19    HPa6219  
                 20                  2              20    HPa6220

Remember that the epoch dataframe index can be used as keys, which can be useful if we want to access a particular epoch.

In [10]:
[tetrode_info[epoch_key]
 for epoch_key in epoch_info.loc[
     (epoch_info.type == 'sleep') & (epoch_info.day == 8)].index]

[                                 area  depth descrip  numcells animal  day  \
 animal day epoch tetrode_number                                              
 HPa    8   1     1                CA1    113  riptet        12    HPa    8   
                  2                CA1    121     NaN         0    HPa    8   
                  3                CA1     90  CA1Ref         0    HPa    8   
                  4                CA1    116  riptet        15    HPa    8   
                  5                CA1    116  riptet         0    HPa    8   
                  6                CA1    110  riptet         0    HPa    8   
                  7                CA1    114  riptet         0    HPa    8   
                  8               iCA1    114  riptet         0    HPa    8   
                  9               iCA1    100  riptet         0    HPa    8   
                  10              iCA1     96     NaN         0    HPa    8   
                  11              iCA1    106  ripte

#### `make_neuron_dataframe`

The neuron dataframe is set up similarly to the tetrode dataframe. It is a dictionary of pandas dataframes, where the keys are the epoch tuples (Animal, Day, Epoch) and the values are the pandas dataframes. The dataframes give descriptive information about the number of spikes, the average spiking rate, which brain area, etc.

In [11]:
from src.data_processing import make_neuron_dataframe
neuron_info = make_neuron_dataframe(ANIMALS)

list(neuron_info.keys())

[('HPa', 2, 1),
 ('HPa', 8, 2),
 ('HPa', 7, 5),
 ('HPb', 6, 4),
 ('HPb', 6, 2),
 ('HPa', 2, 2),
 ('HPa', 8, 3),
 ('HPa', 7, 4),
 ('HPc', 2, 4),
 ('HPb', 1, 2),
 ('HPa', 2, 3),
 ('HPb', 3, 4),
 ('HPc', 2, 5),
 ('HPa', 2, 4),
 ('HPa', 8, 1),
 ('HPa', 5, 5),
 ('HPa', 2, 5),
 ('HPb', 3, 2),
 ('HPa', 5, 4),
 ('HPc', 4, 4),
 ('HPa', 2, 6),
 ('HPb', 3, 3),
 ('HPc', 4, 5),
 ('HPa', 2, 7),
 ('HPa', 8, 4),
 ('HPb', 7, 5),
 ('HPb', 7, 2),
 ('HPa', 3, 5),
 ('HPa', 8, 5),
 ('HPb', 5, 4),
 ('HPa', 5, 1),
 ('HPa', 3, 4),
 ('HPb', 7, 4),
 ('HPa', 3, 3),
 ('HPc', 3, 1),
 ('HPb', 8, 4),
 ('HPa', 5, 3),
 ('HPc', 4, 1),
 ('HPa', 7, 2),
 ('HPb', 5, 1),
 ('HPb', 5, 3),
 ('HPa', 5, 2),
 ('HPa', 3, 1),
 ('HPb', 6, 1),
 ('HPb', 4, 2),
 ('HPc', 3, 3),
 ('HPc', 4, 3),
 ('HPc', 3, 2),
 ('HPb', 8, 3),
 ('HPc', 3, 5),
 ('HPb', 5, 5),
 ('HPb', 8, 2),
 ('HPb', 2, 5),
 ('HPc', 3, 4),
 ('HPb', 8, 1),
 ('HPb', 2, 4),
 ('HPc', 6, 4),
 ('HPc', 8, 2),
 ('HPb', 2, 2),
 ('HPc', 6, 5),
 ('HPb', 4, 5),
 ('HPa', 6, 4),
 ('HPc',

We can access the neuron dataframe for a particular epoch in the same way as the tetrodes

In [12]:
epoch_key = ('HPa', 6, 2)
neuron_info[epoch_key]

area                   csi  \
animal day epoch tetrode_number neuron_number                               
HPa    6   2     1              1               CA1  0.004048582995951417   
                                2               CA1   0.11722488038277512   
                                3               CA1   0.04294478527607362   
                                4               CA1   0.10416666666666667   
                                5               CA1   0.09194977843426884   
                                6               CA1   0.09351432880844646   
                 4              1               CA1   0.20634920634920634   
                                2               CA1   0.08823529411764706   
                                3               CA1  0.023166023166023165   
                                4               CA1   0.15186028853454822   
                                5               CA1   0.15492957746478872   
                                6               CA1   0.04779411764705882   
                                7               NaN                   NaN   
                                8               NaN                   NaN   
                 5              1               CA1   0.26758409785932724   
                                2               CA1   0.09761824890976183   
                                3               NaN                   NaN   
                                4               CA1    0.1951219512195122   
                                5               CA1   0.06997929606625258   
                 8              1              iCA1   0.04825737265415549   
                 9              1              iCA1   0.08766233766233766   
                 10             1              iCA1   0.06779661016949153   
                 11             1              iCA1    0.1391057487579844   
                 12             1              iCA1   0.03225806451612903   
                 14             1              iCA1   0.08710801393728224   
                 17             1               PFC   0.20539862821742016   
                 18             1               PFC  0.006277463904582548   

                                                          meanrate  \
animal day epoch tetrode_number neuron_number                        
HPa    6   2     1              1               0.4099585062240664   
                                2              0.34688796680497924   
                                3               0.4058091286307054   
                                4              0.15933609958506223   
                                5               2.2473029045643154   
                                6               0.5502074688796681   
                 4              1              0.15684647302904564   
                                2              0.11286307053941909   
                                3               0.2149377593360996   
                                4               1.0929460580912862   
                                5              0.23568464730290456   
                                6              0.45145228215767635   
                                7                              NaN   
                                8                              NaN   
                 5              1               0.5427385892116182   
                                2                2.473858921161826   
                                3                              NaN   
                                4               0.5443983402489626   
                                5                2.004149377593361   
                 8              1               0.3095435684647303   
                 9              1               0.5112033195020746   
                 10             1               0.2937759336099585   
                 11             1               1.1692946058091287   
                 12             1               0.3087136929460

#### `get_spike_indicator_dataframe`

The spike indicator dataframe is a dataframe where each row corresponds to the recording timestamp and `is_spike` is an indicator function where 1 indicates a spike has occurred at that timestamp and 0 indicates no spike has occurred at that timestamp. We can access the spike indicator dataframe for a particular neuron by using the neuron key (Animal, Day, Epoch, Tetrode, Neuron).


In [13]:
from src.data_processing import get_spike_indicator_dataframe

neuron_key = ('HPa', 6, 2, 1, 4)
get_spike_indicator_dataframe(neuron_key, ANIMALS)

,is_spike
time,
4122.992000,0
4122.992667,0
4122.993333,0
4122.994000,0
4122.994667,0
4122.995333,0
4122.996000,0
4122.996667,0
4122.997333,0


This information can be obtained from the neuron dataframe. The index of the neuron dataframe gives the key.

In [14]:
neuron_info[epoch_key].index.tolist()

[('HPa', 6, 2, 1, 1),
 ('HPa', 6, 2, 1, 2),
 ('HPa', 6, 2, 1, 3),
 ('HPa', 6, 2, 1, 4),
 ('HPa', 6, 2, 1, 5),
 ('HPa', 6, 2, 1, 6),
 ('HPa', 6, 2, 4, 1),
 ('HPa', 6, 2, 4, 2),
 ('HPa', 6, 2, 4, 3),
 ('HPa', 6, 2, 4, 4),
 ('HPa', 6, 2, 4, 5),
 ('HPa', 6, 2, 4, 6),
 ('HPa', 6, 2, 4, 7),
 ('HPa', 6, 2, 4, 8),
 ('HPa', 6, 2, 5, 1),
 ('HPa', 6, 2, 5, 2),
 ('HPa', 6, 2, 5, 3),
 ('HPa', 6, 2, 5, 4),
 ('HPa', 6, 2, 5, 5),
 ('HPa', 6, 2, 8, 1),
 ('HPa', 6, 2, 9, 1),
 ('HPa', 6, 2, 10, 1),
 ('HPa', 6, 2, 11, 1),
 ('HPa', 6, 2, 12, 1),
 ('HPa', 6, 2, 14, 1),
 ('HPa', 6, 2, 17, 1),
 ('HPa', 6, 2, 18, 1)]

Like the epoch and tetrode dataframe, this allows us to filter for certain attributes (like if we want neurons in a certain brain area) and select only those neurons. For example, if we want only CA1 neurons:

In [15]:
neuron_info[epoch_key].query('area == "CA1"')

area                   csi  \
animal day epoch tetrode_number neuron_number                              
HPa    6   2     1              1              CA1  0.004048582995951417   
                                2              CA1   0.11722488038277512   
                                3              CA1   0.04294478527607362   
                                4              CA1   0.10416666666666667   
                                5              CA1   0.09194977843426884   
                                6              CA1   0.09351432880844646   
                 4              1              CA1   0.20634920634920634   
                                2              CA1   0.08823529411764706   
                                3              CA1  0.023166023166023165   
                                4              CA1   0.15186028853454822   
                                5              CA1   0.15492957746478872   
                                6              CA1   0.04779411764705882   
                 5              1              CA1   0.26758409785932724   
                                2              CA1   0.09761824890976183   
                                4              CA1    0.1951219512195122   
                                5              CA1   0.06997929606625258   

                                                          meanrate  \
animal day epoch tetrode_number neuron_number                        
HPa    6   2     1              1               0.4099585062240664   
                                2              0.34688796680497924   
                                3               0.4058091286307054   
                                4              0.15933609958506223   
                                5               2.2473029045643154   
                                6               0.5502074688796681   
                 4              1              0.15684647302904564   
                                2              0.11286307053941909   
                                3               0.2149377593360996   
                                4               1.0929460580912862   
                                5              0.23568464730290456   
                                6              0.45145228215767635   
                 5              1               0.5427385892116182   
                                2                2.473858921161826   
                                4               0.5443983402489626   
                                5                2.004149377593361   

                                               neuron_number numspikes  \
animal day epoch tetrode_number neuron_number                            
HPa    6   2     1              1                          1       494   
                                2                          2       418   
                                3                          3       489   
                                4                          4       192   
                                5                          5      2708   
                                6                          6       663   
                 4              1                          1       189   
                                2                          2       136   
                                3                          3       259   
                                4                          4      1317   
                                5                          5       284   
                                6                          6       544   
                 5              1                          1       654   
                                2                          2      2981   
                                4                          4       656   
                                5                          5      2415   

                                                        propbursts  \
animal day ep

We can get the keys for CA1 neurons only:

In [16]:
neuron_info[epoch_key].query('area == "CA1"').index.tolist()

[('HPa', 6, 2, 1, 1),
 ('HPa', 6, 2, 1, 2),
 ('HPa', 6, 2, 1, 3),
 ('HPa', 6, 2, 1, 4),
 ('HPa', 6, 2, 1, 5),
 ('HPa', 6, 2, 1, 6),
 ('HPa', 6, 2, 4, 1),
 ('HPa', 6, 2, 4, 2),
 ('HPa', 6, 2, 4, 3),
 ('HPa', 6, 2, 4, 4),
 ('HPa', 6, 2, 4, 5),
 ('HPa', 6, 2, 4, 6),
 ('HPa', 6, 2, 5, 1),
 ('HPa', 6, 2, 5, 2),
 ('HPa', 6, 2, 5, 4),
 ('HPa', 6, 2, 5, 5)]

And then get the spike indicator data for those neurons:

In [17]:
pd.concat(
    [get_spike_indicator_dataframe(neuron_key, ANIMALS)
     for neuron_key in neuron_info[epoch_key].query('area == "CA1"').index], axis=1)

,is_spike,is_spike,is_spike,is_spike,is_spike,is_spike,is_spike,is_spike,is_spike,is_spike,is_spike,is_spike,is_spike,is_spike,is_spike,is_spike
time,,,,,,,,,,,,,,,,
4122.992000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4122.992667,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4122.993333,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4122.994000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4122.994667,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4122.995333,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4122.996000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4122.996667,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4122.997333,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


If we want the numpy array, use `.values`

In [18]:
pd.concat(
    [get_spike_indicator_dataframe(neuron_key, ANIMALS)
     for neuron_key in neuron_info[epoch_key].query('area == "CA1"').index], axis=1).values

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

#### `get_interpolated_position_dataframe`

This dataframe gives information about the animal's position during an epoch. Like the tetrode and neuron dataframes, it is a dictionary of dataframes where the keys correspond to the epoch (Animal, Day, Epoch).

In [19]:
from src.data_processing import get_interpolated_position_dataframe

epoch_key = ('HPa', 6, 2)
get_interpolated_position_dataframe(epoch_key, ANIMALS)

,trajectory_category_ind,trajectory_direction,trajectory_turn,trial_number,linear_distance,x_position,y_position,head_direction,speed,linear_position
time,,,,,,,,,,
4122.992000,2.0,Inbound,Right,1.0,3.954611,87.594914,95.793044,-1.662769,28.374497,-4.247499
4122.992667,2.0,Inbound,Right,1.0,3.954611,87.594914,95.793044,-1.662769,28.374497,-4.247499
4122.993333,2.0,Inbound,Right,1.0,3.954611,87.594914,95.793044,-1.662769,28.374497,-4.247499
4122.994000,2.0,Inbound,Right,1.0,3.954611,87.594914,95.793044,-1.662769,28.374497,-4.247499
4122.994667,2.0,Inbound,Right,1.0,3.954611,87.594914,95.793044,-1.662769,28.374497,-4.247499
4122.995333,2.0,Inbound,Right,1.0,3.954611,87.594914,95.793044,-1.662769,28.374497,-4.247499
4122.996000,2.0,Inbound,Right,1.0,3.954611,87.594914,95.793044,-1.662769,28.374497,-4.247499
4122.996667,2.0,Inbound,Right,1.0,3.954611,87.594914,95.793044,-1.662769,28.374497,-4.247499
4122.997333,2.0,Inbound,Right,1.0,3.954611,87.594914,95.793044,-1.662769,28.374497,-4.247499
